In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install wget -q
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm
import ast, wget
from glob import glob
from time import sleep
from bs4 import BeautifulSoup
#change file location according to where this notebook is located.
data_folder = '/content/drive/MyDrive/pop_pharmacogenomics/onsides_intl/uk_drug_label/raw_data/'
api = 'bbf40496-e20e-4f79-9636-b63d4a3deea7'

  Preparing metadata (setup.py) ... done


This notebook should iterate through the medicines.org.uk/emc website to download the html drug label files into a sub-folder "raw/" in the data_folder above.

##Scrape MHRA website

In [ ]:
url_df = pd.read_csv(data_folder+'product_urls.csv')
url_df['product-soup'] = url_df['product-soup'].apply(lambda x: BeautifulSoup(x, 'html.parser'))
url_df['product-class'] = url_df['product-soup'].apply(lambda x: x.find(class_='left').text)
url_df['product-url'] = url_df['product-soup'].apply(lambda x: x.find(class_='right').find('a', href=True)['href'])
url_df = url_df[url_df['product-class'] == 'SPC'].drop_duplicates()
url_df['product-id'] = url_df.index
url_df.to_csv(data_folder+'product_urls_updated.csv', index=False)
url_df.head(1)

,product-name,product-soup,product-class,product-url,product-id
3,ADJUVANTED QUADRIVALENT INFLUENZA VACCINE SEQI...,"[[[SPC]], [[<p class=""title"">ADJUVANTED QUADRI...",SPC,https://mhraproducts4853.blob.core.windows.net...,3


In [ ]:
for i, row in tqdm(url_df.iterrows()):
  wget.download(row['product-url'], out=data_folder+'pdf/{}.txt'.format(row['product-id']))
  sleep(1)

4809it [2:57:17,  2.21s/it]


In [ ]:
url_df = pd.read_csv(data_folder+'product_urls.csv')
print(url_df.shape[0])
url_df['product-soup'] = url_df['product-soup'].apply(lambda x: BeautifulSoup(x, 'html.parser'))
url_df['product-class'] = url_df['product-soup'].apply(lambda x: x.find(class_='left').text)
url_df['product-url'] = url_df['product-soup'].apply(lambda x: x.find(class_='right').find('a', href=True)['href'])
url_df = url_df[url_df['product-class'] == 'SPC'].drop_duplicates()
url_df['product-id'] = url_df.index
url_df.head(1)

40812


,product-name,product-soup,product-class,product-url,product-id
3,ADJUVANTED QUADRIVALENT INFLUENZA VACCINE SEQI...,"[[[SPC]], [[<p class=""title"">ADJUVANTED QUADRI...",SPC,https://mhraproducts4853.blob.core.windows.net...,3


In [ ]:
url_df.shape[0]

4809

## Extract Text from PDF files

In [ ]:
!pip install pdfminer.six
from pdfminer.high_level import extract_text
from io import StringIO
from pdfminer.high_level import extract_text_to_fp
from pdfminer.layout import LAParams
import json, re

In [ ]:
subtitles = ['NAME OF THE MEDICINAL PRODUCT','QUALITATIVE AND QUANTITATIVE COMPOSITION','PHARMACEUTICAL FORM',
'Therapeutic indications','Posology and method of administration','Contraindications',
'Special warnings and precautions for use','Interaction with other medicinal products and other forms of interaction',
'Fertility, pregnancy and lactation','Effects on ability to drive and use machines','Undesirable effects',
'Overdose','Pharmacodynamic properties','Pharmacokinetic properties','Preclinical safety data','List of excipients',
'Incompatibilities','Shelf life','Special precautions for storage','Nature and contents of container',
'Special precautions for disposal','MARKETING AUTHORISATION HOLDER','MARKETING AUTHORISATION NUMBER(S)',
'DATE OF FIRST AUTHORISATION/RENEWAL OF THE AUTHORISATION','DATE OF REVISION OF THE TEXT']

In [ ]:
files = glob(data_folder+'pdf/*')
for f in tqdm(files):
  name = f.split('/')[-1].replace('.txt','')
  try:
    t = extract_text(f, laparams=LAParams())
    # Initialize a dictionary to store sections of the text
    sections = {}
    # Split the raw text into sections based on the subtitles
    for i in range(len(subtitles)-1):
        # Create a regex pattern to find the text between two subtitles
        pattern = f'({subtitles[i]})((.|\n)*?)({subtitles[i+1]})'
        # Search for the pattern in the raw text
        match = re.search(pattern, t)
        # If a match is found, add it to the dictionary (trim whitespace and remove subtitle)
        if match:
            content = match.group(2).strip()
            sections[subtitles[i]] = content
    # Special handling for the last subtitle (goes until the end of the document)
    match = re.search(f'({subtitles[-1]})((.|\n)*)', t)
    if match:
        content = match.group(2).strip()
        sections[subtitles[-1]] = content
    # Write the dictionary to a JSON file
    with open(data_folder+'json/{}.json'.format(name), 'w') as file:
        json.dump(sections, file)
  except:
    pass

100%|██████████| 4809/4809 [1:10:31<00:00,  1.14it/s]


## Extract Tables

In [ ]:
!pip install PyPDF2 wget pypdf tabula-py pdfminer camelot-py[cv] -q
import tabula, camelot
import pypdf
import PyPDF2
import wget
from pypdf import PdfReader

In [ ]:
files = glob(data_folder+'pdf/*')
for f in tqdm(files):
  name = f.split('/')[-1].replace('.txt','')
  try:
    tables = tabula.read_pdf(f,pages='all', silent=True)
    for j, table in enumerate(tables):
      columns = ', '.join(table.columns.tolist()).lower()
      if 'system organ class' in columns or 'adverse reaction' in columns or 'frequency' in columns:
        table['name'] = name
        table.to_csv(data_folder+'tables/{drug}_{j}.csv'.format(drug=name, j=j))
  except:
    pass

 60%|██████    | 2889/4809 [3:31:46<2:19:24,  4.36s/it]

In [ ]:
files = glob(data_folder+'pdf/*')
ades = pd.DataFrame()
for file in tqdm(files):
  name = file.split('/')[-1].replace('.txt','')
  #extract tables
  try:
    tables = tabula.read_pdf(file,pages='all', silent=True)
    for table in tables:
      columns = [i.lower() for i in table.columns]
      if 'system organ class' in list(columns):
        if 'adverse reaction' in columns and 'frequency' in columns:
          table['name'] = name
          ades = ades.append(table)
        else:
          print('oops')
          break
  except:
    continue

  0%|          | 3/4809 [00:10<4:48:34,  3.60s/it]

oops


  0%|          | 8/4809 [00:28<5:05:38,  3.82s/it]

oops


  0%|          | 11/4809 [00:42<6:25:22,  4.82s/it]

...

  5%|▌         | 252/4809 [17:07<5:06:00,  4.03s/it]

oops


  5%|▌         | 254/4809 [17:14<4:41:20,  3.71s/it]

oops


  5%|▌         | 257/4809 [17:20<3:00:09,  2.37s/it]

In [ ]:
ades_df = pd.DataFrame(ades)
ades_df.to_csv(data_folder+'raw_ade_tables.csv', index=False)
ades_df.head(1)